In [11]:
import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as plt


In [17]:
def F_nu(t_p,nu_p,F_nup):
    m = 0.88 #using stAandard model
    p = 3.0 #using standard model
    a = (2*m)+0.5
    b = (p+5-(6*m))/2
    t=t_p
    nu=nu_p

    F_nu = F_nup*1.582*(t/t_p)**a*(nu/nu_p)**(5/2)*(1-np.exp(-(t/t_p)**(-(a+b))*(nu/nu_p)**(-(p+4)/2))
        print F_nu(t_p nu,F_nup )
                                                    
    return F_nu(1224.288326 , 8,  0.0272)                                           

SyntaxError: invalid syntax (<ipython-input-17-cb7ec5160ef5>, line 11)

In [ ]:
F_p =
nu_p =
for F in F_p:
     F = F_p*1.582*(t/t_p)**a*(nu/nu_p)**(5/2)*(1-np.exp(-(t/t_p)**(-(a+b))*(nu/nu_p)**(-(p+4)/2))
        print F_(t_p nu,F_p )
    for nu in nu_p:
    #calculate SSA
  #calculate Mass loss rate, Radius
               alpha = 1  #fraction of charged particles and magnetic field (epsilon_e/epsilon_b)
f =  0.5   # filling factor (fraction of the emiting region)
D =  808.4  # in Mpc distance to the SN

R = 4.0e14*(alpha)**(-1/19)*(f/0.5)**(-1/19)*(F_p)**(9/19)*(D)**(18/19)*(nu_p/5)**(-1)

# Magnetic field flux
B = 1.1*(alpha)**(-4/19)*(f/0.5)**(-4/19)*(F_p)**(-2/19)*(D)**(-4/19)*(nu_p/5)

#Internal energy of the emitting material
epsilon_b = 0.1
E = (1/epsilon_b)*((B**2)/(8* 3.142))*((4*3.142*f*R**3)/3)

#expansion velocity in km/s
v = (R/t)*1.1574e-10

#pre-explosion mass-loss in 1e-5 solar mass per year
v_w = 1000  #wind speed in km/s
M = 1.0*(alpha)**(-8/19)*epsilon_b*(f)**(-8/19)*(F_p)**(-4/19)*(D)**(-4/19)*(nu_p/5)*(t_p/10)*(v_w/1000)

        #calculate tau_ff
       M_cgs =  #mass loss rate in cgs units
Z_ave = 5.4  # Average metallicity 1= pure H. 5.4 for a massive star
miu = 1.9 # mean molecular weight of electrons. 1= pure H  1.9 is for a massive star.
#nu =  #frequency array that defines your SED. In GHz
#R =   #radius in cm
vw_cgs = 1000 * 10^5 # assumed wind velocity in cgs (cm/s). Take this as 1000 * 10^5 for now. (i.e. 1000 km/s in cgs)
T = 10^4 # temperature of the material absorbing in K.  10^4 is a good starting point. 
                                         
   tau_ff = 2.021e25*M_cgs**2*Z_ave/(miu**2*nu**(2.1)*R**3*vw_cgs**2*T**(1.35))  
       F_ff = F*np.exp(-tau_ff)   
                                      